# Customer Segmentation 

This notebook will focus on the creation, deployment, 
monitoring and management of a machine learning model for performing 
customer segmentation. 

We will be using an e-commerce dataset detailing actual purchases made by  ∼ 4000 customers over a period of one year (from 2010/12/01 to 2011/12/09). 

In this notebook we will: 

- Explore a subset of the dataset
- Train several models on the pre-processed dataset
- Deploy trained models to Seldon 
- Train an anchor tabular explainer and update Seldon deployment with explainer
- Train an outlier detector (variational autoencoder) and update deployment 

### Prerequisites
Initially, we install some additional packages which do not come out of the box with our Colab environment, and then import all of the relevant packages. 

In [34]:
!pip install seldon-deploy-sdk==1.3.0
!pip install alibi
!pip install alibi-detect
!pip install fsspec
!pip install gcsfs
!pip install dill

In [ ]:
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.svm import SVC
from sklearn import neighbors, linear_model, svm, tree, ensemble
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import accuracy_score
import numpy as np
import pickle 
import joblib
import dill
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense, InputLayer
from alibi.explainers import AnchorTabular
from alibi_detect.datasets import fetch_kdd
from alibi_detect.models.tensorflow.losses import elbo
from alibi_detect.od import OutlierVAE
from alibi_detect.cd import TabularDrift
from alibi_detect.utils.data import create_outlier_batch
from alibi_detect.utils.fetching import fetch_detector
from alibi_detect.utils.saving import save_detector, load_detector
from alibi_detect.utils.visualize import plot_instance_score, plot_feature_outlier_tabular, plot_roc
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, OutlierDetectorApi, DriftDetectorApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

We then download our pre-processed dataset to save time in getting to the exciting parts:

In [ ]:
!gsutil -m cp -r gs://tom-seldon-examples/retail-workshop/data .

### Data Exploration

Typically e-commerce datasets are proprietary and consequently hard to find among publicly available data. However, The UCI Machine Learning Repository has curated a retail dataset containing actual transactions from 2010 and 2011. The dataset is maintained on their site, where it can be found by the title "Online Retail". The following description is provided via Kaggle: 

"This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."

We will first explore the dataset and then train models on a pre-processed version of the dataset that was transformed through following the fantastic work of Fabien Daniel whose methods are detailed in [this notebook](https://www.kaggle.com/fabiendaniel/customer-segmentation).

Lets dive in and explore a subset of the dataset:

In [ ]:
df_initial = pd.read_csv('data/data.csv', encoding="ISO-8859-1", nrows=3000,
                         dtype={'CustomerID': str,'InvoiceID': str})

print('Dataframe dimensions:', df_initial.shape)

In [ ]:
df_initial['InvoiceDate'] = pd.to_datetime(df_initial['InvoiceDate'])

# gives some infos on columns types and numer of null values
tab_info=pd.DataFrame(df_initial.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()/df_initial.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
display(tab_info)

# show first lines
display(df_initial[:5])

The dataset contains the following features: 

**InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.

**StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.

**Description**: Product (item) name. Nominal.

**Quantity**: The quantities of each product (item) per transaction. Numeric.

**InvoiceDate**: Invice Date and time. Numeric, the day and time when each transaction was generated.

**UnitPrice**: Unit price. Numeric, Product price per unit in sterling.

**CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.

**Country**: Country name. Nominal, the name of the country where each customer resides.

We will be training models using a transformed version of this dataset. Rather than detailing each individual item bought, the transformed dataset details the spending behavoir of each individual customer across different categories of products. We don't have any product categories in the original dataset so we create this feature through analysing keywords used within product descriptions and applying k-means clustering to create a finite number of product clusters. We find in our analysis that 5 clusters is a suitable number to seperate products without creating clusters containing too few products (calculated using silhouette score). The following wordclouds demonstrate the splits we generate:

<img src="https://raw.githubusercontent.com/ribenamaplesyrup/seldon-retail-workshop/main/assets/download.png">

We see that cluster no1 contains words we might associate more with gifts ('christmas', 'card', 'wrap', 'decoration') and no3 more with luxury goods ('lace', 'necklace'), although some of the other clusters are less differentiated.

Based on customer spending across each of these categories of product, we can similarly use k-means to cluster customers into different segments. Performing similar analysis to how we clustered product categories, we find 11 clusters is the most suitable for segmenting customers within this dataset.

In [ ]:
columns = ['mean', 'categ_0', 'categ_1', 'categ_2', 'categ_3', 'categ_4' ] # change to feature_names?
class_names = ['0','1','2','3','4','5','6','7','8','9','10']
X_train = np.load('data/X_train.npy')
Y_train = np.load('data/Y_train.npy')
X_test = np.load('data/X_test.npy')
Y_test = np.load('data/Y_test.npy')

The transformed dataset contains the following features specific to each unique customer:

**mean**: mean spend across all transactions

**categ_0**: percentage of total spend on products within category 0

**categ_1**: percentage of total spend on products within category 1

**categ_2**: percentage of total spend on products within category 2

**categ_3**: percentage of total spend on products within category 3

**categ_4**: percentage of total spend on products within category 4

**customer_category**: integer signifying which of the 11 categories the customer aligns most with

In [ ]:
customers = pd.DataFrame(Y_train, columns=['customer_category']).astype(dtype=int)
spending = pd.DataFrame(X_train, columns = columns)
transformed_dataset = pd.concat([spending, customers], axis=1)
transformed_dataset

We can explore the range of values across each of the features within our dataset:

In [ ]:
transformed_dataset.max(axis=0)

In [ ]:
transformed_dataset.min(axis=0)

## Model Training

In this section we will adjust a classifier that will classify consumers in the different client categories. 

In [ ]:
# Standardise the dataset
mean, stdev = X_train.mean(axis=0), X_train.std(axis=0)
X_train = (X_train - mean) / stdev
X_test = (X_test - mean) / stdev

### Logistic Regression

In [ ]:
lr = linear_model.LogisticRegression(max_iter=4000)
lr.fit(X_train, Y_train)

In [ ]:
accuracy_score(Y_test, lr.predict(X_test))

### Gradient Boosting Classifier 

In [ ]:
gb=ensemble.GradientBoostingClassifier()
gb.fit(X = X_train, y = Y_train)

In [ ]:
accuracy_score(Y_test, gb.predict(X_test))

In [ ]:
models = ['lr', 'gb']

for model in models:
    if not os.path.exists('models/' + model):
        os.makedirs('models/' + model)

In [ ]:
filename = 'models/lr/model.joblib'
joblib.dump(lr, filename)

In [ ]:
filename = 'models/gb/model.joblib'
joblib.dump(gb, filename)

### Push model artefacts to GCP

To push models to GCP you will create a unique folder with your name. Uncomment the command and replace "YOUR NAME" with your name.

In [ ]:
!gsutil cp models/lr/model.joblib gs://tom-seldon-examples/retail-workshop/models/<YOUR NAME>/lr/model.joblib
!gsutil cp models/gb/model.joblib gs://tom-seldon-examples/retail-workshop/models/<YOUR NAME>/gb/model.joblib

### Deploy models to Seldon

We can now deploy our model to the dedicated Seldon Deploy cluster which we have configured for this workshop. To do so we will interact with the Seldon Deploy SDK.

First, setting up the configuration and authentication required to access the cluster. Make sure to fill in the SD_IP variable to be the same as the cluster you are using.

In [ ]:
SD_IP = "34.91.77.143"

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.username = "admin@seldon.io"
config.password = "12341234"
config.auth_method = 'password_grant'

def auth():
    auth = OIDCAuthenticator(config)
    config.access_token = auth.authenticate()
    api_client = ApiClient(config)
    return api_client

Now we have configured the IP correctly as well as setup our authentication function we can desrcibe the deployment we would like to create.

You will need to fill in the DEPLOYMENT_NAME, NAMESPACE, and the MODEL_LOCATION, the rest of the deployment description has been templated for you.

You will need to create a unique deployment name. A good example format would be "YOUR NAME" + "MODEL" so "sgreaves-lr" in my case (be careful not to use any upper-case letters or other characters like "_". 

In [ ]:
YOUR_NAME = "<YOUR NAME>"
MODEL_NAME = "<MODEL NAME>"

DEPLOYMENT_NAME = f"{YOUR_NAME}-{MODEL_NAME}"
MODEL_LOCATION = f"gs://tom-seldon-examples/retail-workshop/models/{YOUR_NAME}/lr"

In [ ]:
NAMESPACE = "default"

PREPACKAGED_SERVER = "SKLEARN_SERVER"

CPU_REQUESTS = "0.1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "0.1"
MEMORY_LIMITS = "1Gi"

mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "implementation": PREPACKAGED_SERVER,
                    "modelUri": MODEL_LOCATION,
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

We can now invoke the `SeldonDeploymentsApi` and create a new Seldon Deployment. 

Time for you to get your hands dirty. You will use the Seldon Deploy SDK to create a new Seldon deployment. You can find the reference documentation [here](https://github.com/SeldonIO/seldon-deploy-sdk/blob/master/python/README.md). 

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

Our model should now be running as a fully fledged microservice. We can now log into Seldon Deploy and test our deployment: 

* URL: http://34.91.77.143/seldon-deploy/
* Username: admin@seldon.io
* Password: 12341234


In [ ]:
# test model with this request: 
{"data": {"ndarray": [[-0.43831067,  0.23457404, -0.17366155,  0.26935668,  0.13463672, -0.55234561]]}}

## Explainer

Next, we shall train an explainer to glean deeper insights into the decisions being made by our model. We will make use of the Anchors algorithm, which has a [production grade implementation available](https://docs.seldon.io/projects/alibi/en/stable/methods/Anchors.html) using the Seldon Alibi Explain library. 

The algorithm provides model-agnostic (black box) and human interpretable explanations suitable for classification models applied to images, text and tabular data. The idea behind anchors is to explain the behaviour of complex models with high-precision rules called anchors. These anchors are locally sufficient conditions to ensure a certain prediction with a high degree of confidence. Anchor algorithms incorporate coverage, which is the region the explanation applies within and are optimised to maximize coverage.

As an example of anchors for tabular data, if we want to predict whether a person makes less or more than £50,000 per year based on the person’s characteristics including age (continuous variable) and marital status (categorical variable), then the following would be a potential anchor: Hugo makes more than £50,000 because he is married and his age is between 35 and 45 years.

The first step will be to write a simple prediction function which the explainer can call in order to query our logistic regression model.

In [ ]:
predict_fn = lambda x: lr.predict(x)

We then initialise our Anchor explainer, using the AnchorTabular flavour provided by Alibi due to our data modality.

In [ ]:
explainer = AnchorTabular(predict_fn, columns)

We will now train our explainer. We need to specify percentages that our model will use to discretize continuous numerical data into intervals. Generally we observe that if we choose smaller percentages (finer discretization), this will lead to longer runtime and potentially higher precision. 

In [ ]:
explainer.fit(X_train, disc_perc=(25, 50, 75))

We can now test our explainer by generating a prediction.

In [ ]:
idx = 0
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

In [ ]:
explanation = explainer.explain(X_test[idx], threshold=0.9)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

We use dill to serialise our explainer:

In [ ]:
with open("models/lr/explainer.dill", "wb") as model_f:
        dill.dump(explainer, model_f)

We will not push our explainer to GCP as explainer files can be quite large (100MB+) so instead we will deploy a pretrained explainer!


## Deployment

We can now deploy an explainer alongside our model:

In [ ]:
EXPLAINER_TYPE = "AnchorTabular"
EXPLAINER_URI = "gs://tom-seldon-examples/retail-workshop/pretrained/lr"

explainer_spec = {
                    "type": EXPLAINER_TYPE,
                    "modelUri": EXPLAINER_URI,
                    "containerSpec": {
                        "name": "",
                        "resources": {}
                    }
                }

In [ ]:
mldeployment['spec']['predictors'][0]['explainer'] = explainer_spec
mldeployment

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

## Outlier Detection
You will now setup your outlier detector. This will pick up anomalous data points in an automated fashion. You will use the Seldon Alibi Detect library to configure a Variational Auto Encoder (VAE) outlier detector.

In this case you are going to assume that any instance which is labelled as customer segmentation cluster 1 is an outlier. Clearly, this is a trivial example, as we are using our model to predict the customer segmen. However it demonstrates the flow of outlier detector creation nicely, and provides a set of easy to identify outliers. 

You now generate a dataset of inliers (all other clusters) and outliers (customer cluster 1). 

In [ ]:
df_total = pd.concat([pd.DataFrame(X_train), pd.DataFrame(Y_train, columns=[6])], axis=1)

inliers = df_total[(df_total[6] != 1)]
inliers.drop(labels=6, axis=1, inplace=True)
inliers_np = inliers.to_numpy()

outliers = df_total[df_total[6] == 1]
outliers.drop(labels=6, axis=1, inplace=True)
outliers_np = outliers.to_numpy()

You will then define the architecture of your VAE. The VAE works by attempting to reconstruct the input data which it receives. The VAE first encodes the data in some latent space (in our case a 2 dimensional vector), and then uses a decoder to reconstruct the original input data from the encoding. This forces the VAE to learn a mapping of input data to the latent space, and vice versa. If input data maps poorly to the latent space, and/or maps poorly from latent space to output then it is likely out of the distribution which the VAE was trained upon. Therefore, we can classify it as an outlier.  

![](https://raw.githubusercontent.com/ribenamaplesyrup/seldon-retail-workshop/main/assets/vae_architecture.png "VAE Architecture")

The first step is to define our VAE architecture. You will use TensorFlow Keras to setup the architecture for your encoder and decoder. 

In [ ]:
# define model, initialize, train and save outlier detector
    
n_features = X_train.shape[1]
latent_dim = 2
    
encoder_net = tf.keras.Sequential(
    [
     InputLayer(input_shape=(n_features,)),
     Dense(20, activation=tf.nn.relu),
     Dense(15, activation=tf.nn.relu),
     Dense(7, activation=tf.nn.relu)
     ])

decoder_net = tf.keras.Sequential(
    [
     InputLayer(input_shape=(latent_dim,)),
     Dense(7, activation=tf.nn.relu),
     Dense(15, activation=tf.nn.relu),
     Dense(20, activation=tf.nn.relu),
     Dense(n_features, activation=None)
     ])

Next you will make use of Alibi Detect's OutlierVAE class and instantiate it using the encoder and decoder architecture you defined above. 

You will then call the `fit` method on your outlier detector. To learn how to correctly reconstruct normal data the VAE is fit on only inlier examples initially. In this case you will use your `inliers` set as this does not contain any outlying data points. 

In [ ]:
# initialize outlier detector
od = OutlierVAE(threshold=None,  # threshold for outlier score
                score_type='mse',  # use MSE of reconstruction error for outlier detection
                encoder_net=encoder_net,  # can also pass VAE model instead
                decoder_net=decoder_net,  # of separate encoder and decoder
                latent_dim=latent_dim,
                samples=5) # number of samples drawn during detection for each instance to detect
# train
od.fit(inliers,
       loss_fn=elbo, # Loss function used for training
       cov_elbo=dict(sim=.01), # If using the elbo loss, this is the covariance matrix
       epochs=50,
       verbose=True)

You now need to set the threshold for your outlier detector. This is the score above which any instance will be considered an outlier. 

To do this you can make use of the `infer_threshold` function which Alibi provides. This will take a batch of data with a specified percentage of outliers, you therefore create a function to generate a dataset with a specified percentage of outliers*.

\*Alibi has a comparable function for generating batches of outlier data called `create_outlier_batch` 

In [ ]:
def generate_outlier_batch(perc_outlier=5, inliers=inliers, outliers=outliers, n_samples=1000):
    outlier_idx = int(np.round(n_samples * (perc_outlier / 100), 0))
    inlier_idx = int(np.round(n_samples * (1 - perc_outlier / 100), 0))

    outlier_batch = outliers[:outlier_idx]
    inlier_batch = inliers[:inlier_idx]

    y_combined = np.concatenate([np.ones(len(outlier_batch)), np.zeros(len(inlier_batch))]).astype('int32')

    X_combined = np.concatenate((outlier_batch, inlier_batch))
    return X_combined, y_combined

X_threshold, y_threshold = generate_outlier_batch()

Now inferring the threshold. 

In [ ]:
perc_outlier = 5
od.infer_threshold(X_threshold, threshold_perc=100-perc_outlier)
print('New threshold: {}'.format(od.threshold))

You can now test your threshold by generating a second batch of outlying data, this time with a higher proportion of outliers. 

In [ ]:
X_outlier, y_outlier = generate_outlier_batch(perc_outlier=10)
print(X_outlier.shape, y_outlier.shape)
print('{}% outliers'.format(100 * y_outlier.mean()))

Generating outlier predictions from the new detector using the freshly created outlier batch. 

In [ ]:
od_preds = od.predict(X_outlier, return_instance_score=True)

Visualising the effectiveness of our outlier detector using a confusion matrix. 

In [ ]:
labels = ['normal', 'outlier']

y_pred = od_preds['data']['is_outlier']
f1 = f1_score(y_outlier, y_pred)
print('F1 score: {}'.format(f1))

cm = confusion_matrix(y_outlier, y_pred)
df_cm = pd.DataFrame(cm, index=labels, columns=labels)
sns.heatmap(df_cm, annot=True, cbar=True, linewidths=.5)
plt.show()

And then using a scatter plot and ROC curve. 

In [ ]:
plot_instance_score(od_preds, y_outlier, labels, od.threshold)

In [ ]:
roc_data = {'VAE': {'scores': od_preds['data']['instance_score'], 'labels': y_outlier}}
plot_roc(roc_data)

You can now save your outlier detector locally, and subsequently push to remote storage. 

In [ ]:
save_detector(od, "outlier_detector")

NOTE: To push models to GCP you will create a unique folder with your name. Uncomment the command and replace "YOUR NAME" with your name.

In [ ]:
# Recursive copy this time as the OD is saved as a directory containing all the relevant binaries and parameters. 
!gsutil cp -r outlier_detector gs://tom-seldon-examples/retail-workshop/models/<YOUR NAME>/

Using your freshly uploaded outlier detector you can deploy this alongside your model into Seldon Deploy. Once again you will need to replace 'YOUR NAME' so we're saving our file to correct place:

In [ ]:
OD_URI = f'gs://tom-seldon-examples/retail-workshop/models/{YOUR_NAME}/outlier_detector/'

In [ ]:
OD_NAME = 'vae-od'

od_config = {'deployment': DEPLOYMENT_NAME,
             'deployment_namespace': None,
             'namespace': 'seldon-logs',
             'params': {'drift_batch_size': None,
                        'env_secret_ref': None,
                        'event_source': f'io.seldon.serving.dev-seldondeployment-{DEPLOYMENT_NAME}-outlier',
                        'event_type': 'io.seldon.serving.inference.outlier',
                        'http_port': '8080',
                        'model_name': OD_NAME,
                        'protocol': 'seldon.http',
                        'reply_url': 'http://seldon-request-logger.seldon-logs',
                        'storage_uri': OD_URI,
                        'user_permission': None},
             'prom_scraping': None,
             'url': None}

In [ ]:
od_api = OutlierDetectorApi(auth())
od_api.create_outlier_detector_seldon_deployment(name=DEPLOYMENT_NAME,
                                                 namespace=NAMESPACE,
                                                 outlier_detector=od_config)

You can now test the outlier detector with the following inlier request:

In [ ]:
{"data": {"ndarray": [[-0.43831067,  0.23457404, -0.17366155,  0.26935668,  0.13463672, -0.55234561]]}}

And then with a massive outlier:

In [ ]:
{"data": {"ndarray": [[10,  10, 10,  10,  10, 10]]}}